In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.request import urlretrieve
import sqlalchemy
from sqlalchemy.orm import Session
import json

In [9]:
csv_url = 'https://raw.githubusercontent.com/NationalGalleryOfArt/opendata/main/data/published_images.csv'
ng_df = pd.read_csv(csv_url)
ng_df


,uuid,iiifurl,iiifthumburl,viewtype,sequence,width,height,maxpixels,created,modified,depictstmsobjectid,assistivetext
0,00004dec-8300-4487-8d89-562d0126b6a1,https://api.nga.gov/iiif/00004dec-8300-4487-8d...,https://api.nga.gov/iiif/00004dec-8300-4487-8d...,primary,0.0,2623,4000,640.0,2010-09-07 15:08:48-04,2021-08-26 14:28:24-04,11975,NaN
1,00007f61-4922-417b-8f27-893ea328206c,https://api.nga.gov/iiif/00007f61-4922-417b-8f...,https://api.nga.gov/iiif/00007f61-4922-417b-8f...,primary,0.0,3365,4332,NaN,2013-07-05 15:41:08-04,2021-07-07 10:26:29-04,17387,NaN
2,0000bd8c-39de-4453-b55d-5e28a9beed38,https://api.nga.gov/iiif/0000bd8c-39de-4453-b5...,https://api.nga.gov/iiif/0000bd8c-39de-4453-b5...,primary,0.0,3500,4688,NaN,2013-08-05 14:31:59-04,2021-07-07 10:34:01-04,19245,NaN
3,0000e5a4-7d32-4c2a-97c6-a6b571c9fd71,https://api.nga.gov/iiif/0000e5a4-7d32-4c2a-97...,https://api.nga.gov/iiif/0000e5a4-7d32-4c2a-97...,primary,0.0,2252,3000,NaN,2013-03-18 14:39:55-04,2021-08-31 10:29:14-04,153987,NaN
4,0001668a-dd1c-48e8-9267-b6d1697d43c8,https://api.nga.gov/iiif/0001668a-dd1c-48e8-92...,https://api.nga.gov/iiif/0001668a-dd1c-48e8-92...,primary,0.0,3446,4448,NaN,2014-01-02 14:50:50-05,2021-07-07 11:09:58-04,23830,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
102908,fffd89a6-7250-4743-839d-7022057057f4,https://api.nga.gov/iiif/fffd89a6-7250-4743-83...,https://api.nga.gov/iiif/fffd89a6-7250-4743-83...,primary,0.0,2782,4000,640.0,2012-02-02 15:28:56-05,2021-08-26 15:06:52-04,47930,NaN
102909,fffe6d61-e686-478d-bafb-ddae9a48c4de,https://api.nga.gov/iiif/fffe6d61-e686-478d-ba...,https://api.nga.gov/iiif/fffe6d61-e686-478d-ba...,primary,0.0,4000,1635,NaN,2016-05-10 16:23:54-04,2021-08-05 17:19:27-04,169631,NaN
102910,fffeab03-0f91-49ba-884a-648e6f84598f,https://api.nga.gov/iiif/fffeab03-0f91-49ba-88...,https://api.nga.gov/iiif/fffeab03-0f91-49ba-88...,primary,0.0,3271,2429,NaN,2015-09-22 15:46:19-04,2021-07-08 13:28:29-04,166633,NaN
102911,ffff2faf-fb5a-479c-b5dc-408245cdc1a0,https://api.nga.gov/iiif/ffff2faf-fb5a-479c-b5...,https://api.nga.gov/iiif/ffff2faf-fb5a-479c-b5...,primary,0.0,3454,3465,NaN,2010-11-04 14:31:14-04,2021-08-05 16:29:41-04,39382,NaN


In [14]:
s4_px = ng_df[ng_df['maxpixels'] == 640]
s4_px['iiifthumburl'][0]

'https://api.nga.gov/iiif/00004dec-8300-4487-8d89-562d0126b6a1/full/!200,200/0/default.jpg'

we can use iiifthumbnail column to use it as a base url to save the images. As we want them in a bigger size we will define a short function to replace existing dimensions for desired ones.

In [23]:
def standarize_image_url(url):
    return url.replace('200,200', '843,')

In [16]:
ng_df['iiifthumburl'] = ng_df['iiifthumburl'].apply(standarize_image_url)

In [19]:
ng_df['iiifthumburl'][0]

'https://api.nga.gov/iiif/00004dec-8300-4487-8d89-562d0126b6a1/full/!843,/0/default.jpg'

In [22]:
path_to_folder = '/media/vonzipper/Elements/System Volume Information/nga_art'
for i in range(len(ng_df)):
    urlretrieve(ng_df['iiifthumburl'][i], path_to_folder + '/' + ng_df['uuid'][i] + '.jpg')

In [2]:
path_to_data = '/home/vonzipper/git/opendata/data/'
files = ['locations', 'media_items', 'media_relationships', 'object_associations', 'objects', 'objects_altnums',
'objects_constituents', 'objects_dimensions', 'objects_historical_data', 'objects_terms', 'objects_text_entries', 'preferred_locations', 'preferred_locations_tms_locations', 'published_images']


In [5]:
tabla_df = pd.read_csv(path_to_data + files[0] + '.csv')
tabla_df.to_csv(path_to_data + files[0] + '.csv', index=False, header= False)

In [12]:
nueva_tabla_df = pd.read_csv(path_to_data + files[0] + '.csv')
nueva_tabla_df.iloc[:, 7]

0           NaN
1           NaN
2        1923.0
3        1906.0
4        1920.0
          ...  
18238    1607.0
18239       NaN
18240       NaN
18241       NaN
18242       NaN
Name: Unnamed: 7, Length: 18243, dtype: float64

In [17]:
def convert_to_int(x):
    if x.isnumeric():
        x = int(x)
    return x

In [28]:
for x in nueva_tabla_df.iloc[:,7]:
    
    if x == 'nan':
        x = int(x)
        print(x)
nueva_tabla_df.iloc[:, 7]

0           NaN
1           NaN
2        1923.0
3        1906.0
4        1920.0
          ...  
18238    1607.0
18239       NaN
18240       NaN
18241       NaN
18242       NaN
Name: Unnamed: 7, Length: 18243, dtype: float64

In [31]:
nueva_tabla_df.iloc[:, 7] = nueva_tabla_df.iloc[:, 7].astype('Int64')
nueva_tabla_df.iloc[:, 7]

0        <NA>
1        <NA>
2        1923
3        1906
4        1920
         ... 
18238    1607
18239    <NA>
18240    <NA>
18241    <NA>
18242    <NA>
Name: Unnamed: 7, Length: 18243, dtype: Int64

In [34]:
nueva_tabla_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18243 entries, 0 to 18242
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   9            18243 non-null  int64  
 1   Unnamed: 1   7751 non-null   float64
 2   Anonymous    18243 non-null  object 
 3   Anonymous.1  18243 non-null  object 
 4   Unnamed: 4   16705 non-null  object 
 5   Anonymous.2  15348 non-null  object 
 6   0            18243 non-null  int64  
 7   Unnamed: 7   14894 non-null  Int64  
 8   Unnamed: 8   13033 non-null  float64
 9   Unnamed: 9   14894 non-null  object 
 10  Unnamed: 10  15453 non-null  object 
 11  Other        18243 non-null  object 
 12  anonymous    18243 non-null  object 
dtypes: Int64(1), float64(2), int64(2), object(8)
memory usage: 1.8+ MB


In [42]:
for x in nueva_tabla_df.columns:
    if nueva_tabla_df[x].dtype == 'float64':
        nueva_tabla_df[x] = nueva_tabla_df[x].astype('Int64')

In [43]:
nueva_tabla_df.to_csv(path_to_data + files[0] + '.csv', index=False, header= False)

In [12]:
def save_csv_without_header(df, path):
    with open(path, 'w') as f:
        df.to_csv(f, index=False, header=False, encoding= 'utf-8')

In [30]:
def convert_float_to_int(df):
    for x in df.columns:
        if df[x].dtype == 'float64':
            df[x] = np.floor(pd.to_numeric(df[x], errors='coerce')).astype('Int64')
            
    return df


In [32]:
for x in files:
    df = pd.read_csv(path_to_data + x + '.csv')
    df = convert_float_to_int(df)
    save_csv_without_header(df, path_to_data + x + 'postgres' + '.csv')
    print(x)

locations
media_items
media_relationships
object_associations


/home/vonzipper/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


objects
objects_altnums
objects_constituents
objects_dimensions
objects_historical_data
objects_terms
objects_text_entries
preferred_locations
preferred_locations_tms_locations
published_images


In [17]:
nga_df = pd.read_csv('nga_query_data.csv')
nga_df

,title,url,classification
0,Untitled,https://api.nga.gov/iiif/004bb603-f18c-4ad3-98...,Painting
1,Country House in a Park,https://api.nga.gov/iiif/00209fb1-64a2-4961-9c...,Painting
2,Cuadro 78,https://api.nga.gov/iiif/008624f7-d50e-4aaa-bf...,Painting
3,Gentlemen's Still Life,https://api.nga.gov/iiif/0072e45c-c9ef-48ac-b4...,Painting
4,Seel,https://api.nga.gov/iiif/0087942e-c830-4f71-9f...,Painting
...,...,...,...
3747,Vanitas Still Life,https://api.nga.gov/iiif/ffcbf669-7d12-4f64-a3...,Painting
3748,"John, 4th Earl of Darnley",https://api.nga.gov/iiif/ff6db495-6391-4f75-b6...,Painting
3749,Odalisque,https://api.nga.gov/iiif/ff85b5dc-6632-465f-a1...,Painting
3750,Untitled,https://api.nga.gov/iiif/ffd8f1da-7548-411b-a1...,Painting


In [31]:
nga_df['url'] = nga_df['url'].apply(standarize_image_url)
nga_df['url'][3]

'https://api.nga.gov/iiif/0072e45c-c9ef-48ac-b4dd-b5994cf5546c/full/!843,/0/default.jpg'